## Import required library

In [1]:
import pandas as pd
from transformers import pipeline, MarianMTModel, MarianTokenizer, AutoTokenizer

## Declare function for each LLM

In [2]:
# LLM pre-train for Classification : facebook/bart-large-mnli
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
def classification(labels, text):
    results = classifier(text, candidate_labels=labels)
    return results

# LLM pre-train for Translation from TH to EN : Helsinki-NLP/opus-mt-th-en
def translate_th_en(text):
    model_name = 'Helsinki-NLP/opus-mt-th-en'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    translated = model.generate(**tokenizer([text], return_tensors="pt", padding=True))
    results = tokenizer.decode(translated[0], skip_special_tokens=True)
    return results

# LLM pre-train for Sentiment : cardiffnlp/twitter-xlm-roberta-base-sentiment
def sentiment_predict(text):
    tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
    sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-xlm-roberta-base-sentiment")
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    truncated_text = tokenizer.decode(inputs["input_ids"][0])
    results = sentiment_analyzer(truncated_text)
    return results


## Load Data

In [7]:
df = pd.read_csv('../Data/data.csv')
df['publishedAt'] = pd.to_datetime(df['publishedAt'])
df = df[df['publishedAt'].dt.year >= 2018]
df

,channel_name,channel_id,playlists_id,subscriber_count,video_count,view_count,title,publishedAt,likes,views
0,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,Karakoram Highway ทางหลวงที่มีวิวสวยที่สุดในโล...,2024-05-04 05:00:15+00:00,1408,51377
1,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,เกาะมันนอก นอนบนเกาะแบบส่วนตัว และล่องเรือกลาง...,2024-05-01 11:30:17+00:00,2457,170361
2,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,9 วัน 8 คืน Road Trip สวิตเซอร์แลนด์ และพาทั้ง...,2024-04-29 11:30:29+00:00,1652,75411
3,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,ขับรถเที่ยวรอบเกาะ Okinawa 18 จุด 3 วัน 2 คืน ...,2024-04-27 05:00:30+00:00,4121,311630
4,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,สำรวจที่มาของรังนก วิธีการเก็บ ไปจนถึงการผลิต ...,2024-04-24 11:30:06+00:00,3125,206571
...,...,...,...,...,...,...,...,...,...,...
1722,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,กินอาหารเช้าอาหรับ! มุ่งหน้าสู่ตัวเมืองยะลา [ป...,2020-01-11 15:01:06+00:00,4377,167746
1723,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,ไม่คิดว่าจะเจอเพื่อนเก่า! ปั่นจักรยานวันแรกสาย...,2020-01-08 12:00:07+00:00,5931,191867
1724,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,เลี้ยงขนมเด็กทั้งหมู่บ้าน เครป+ชานมไข่มุก | ปั...,2020-01-06 12:00:05+00:00,4986,150507
1725,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,"คนพากย์ถึงกับขำชื่อทีม! เมื่อต้องพากย์""ทีมไก่ฝ...",2020-01-04 12:00:16+00:00,3821,143820


## Use LLM to translate video Title from Thai to English

In [12]:
english_title = []

for tiltle_th in df['title']:
    result = translate_th_en(tiltle_th)
    english_title.append(result)

df['tiltle_en'] = english_title
df

,channel_name,channel_id,playlists_id,subscriber_count,video_count,view_count,title,publishedAt,likes,views,tiltle_en
0,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,Karakoram Highway ทางหลวงที่มีวิวสวยที่สุดในโล...,2024-05-04T05:00:15Z,1408,51377,Karkoram Highway is the most beautiful highway...
1,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,เกาะมันนอก นอนบนเกาะแบบส่วนตัว และล่องเรือกลาง...,2024-05-01T11:30:17Z,2457,170361,Outland Island lays on a private island and sa...
2,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,9 วัน 8 คืน Road Trip สวิตเซอร์แลนด์ และพาทั้ง...,2024-04-29T11:30:29Z,1652,75411,"Nine days, eight nights, Road Trip Switzerland..."
3,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,ขับรถเที่ยวรอบเกาะ Okinawa 18 จุด 3 วัน 2 คืน ...,2024-04-27T05:00:30Z,4121,311630,"Drive around Okinawa Island, 18, dot, three da..."
4,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,สำรวจที่มาของรังนก วิธีการเก็บ ไปจนถึงการผลิต ...,2024-04-24T11:30:06Z,3125,206571,"Look at the origin of the nest, how it's colle..."
...,...,...,...,...,...,...,...,...,...,...,...
1898,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,ตออีฟ! ซาอุไม่ได้มีแค่ Makkah/Madinah,2018-07-28T07:02:40Z,3311,116869,Toves! Saudis aren't just makah/Madinah.
1899,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,เดินงาน Halal Expo Hatyai 2018,2018-07-22T10:00:02Z,818,35007,Halal Expo Hasiai 2018.
1900,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,Yes I Go trailer,2018-07-20T10:16:20Z,463,28953,"Yes, I went to the trailer."
1901,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,ข้าวยำอบกรอบ รีวิว!,2018-07-17T13:22:11Z,924,30374,"Grilled rice, Review!"


## Use LLM to Classification Video Title (tourism : Yes/No)

In [13]:
tourism_flag = []
category = ['tourism','not tourism']

for tiltle_en in df['tiltle_en']:
    result = classification(category, tiltle_en)
    if result['labels'][0] == 'tourism':
        val = "Yes"
    else:
        val = "No"
    tourism_flag.append(val)
    
df['tourism_flag'] = tourism_flag
df

,channel_name,channel_id,playlists_id,subscriber_count,video_count,view_count,title,publishedAt,likes,views,tiltle_en,tourism_flag
0,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,Karakoram Highway ทางหลวงที่มีวิวสวยที่สุดในโล...,2024-05-04T05:00:15Z,1408,51377,Karkoram Highway is the most beautiful highway...,Yes
1,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,เกาะมันนอก นอนบนเกาะแบบส่วนตัว และล่องเรือกลาง...,2024-05-01T11:30:17Z,2457,170361,Outland Island lays on a private island and sa...,No
2,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,9 วัน 8 คืน Road Trip สวิตเซอร์แลนด์ และพาทั้ง...,2024-04-29T11:30:29Z,1652,75411,"Nine days, eight nights, Road Trip Switzerland...",Yes
3,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,ขับรถเที่ยวรอบเกาะ Okinawa 18 จุด 3 วัน 2 คืน ...,2024-04-27T05:00:30Z,4121,311630,"Drive around Okinawa Island, 18, dot, three da...",Yes
4,Go Went Go,UCqs3pijNrYHZCdl_HGfgTQw,UUqs3pijNrYHZCdl_HGfgTQw,1340000,502,231763874,สำรวจที่มาของรังนก วิธีการเก็บ ไปจนถึงการผลิต ...,2024-04-24T11:30:06Z,3125,206571,"Look at the origin of the nest, how it's colle...",Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
1898,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,ตออีฟ! ซาอุไม่ได้มีแค่ Makkah/Madinah,2018-07-28T07:02:40Z,3311,116869,Toves! Saudis aren't just makah/Madinah.,No
1899,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,เดินงาน Halal Expo Hatyai 2018,2018-07-22T10:00:02Z,818,35007,Halal Expo Hasiai 2018.,No
1900,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,Yes I Go trailer,2018-07-20T10:16:20Z,463,28953,"Yes, I went to the trailer.",No
1901,YES I GO,UCtx3Ac_gLDL5V_OTmgNzHpQ,UUtx3Ac_gLDL5V_OTmgNzHpQ,1330000,523,161564249,ข้าวยำอบกรอบ รีวิว!,2018-07-17T13:22:11Z,924,30374,"Grilled rice, Review!",No


## Save final data

In [18]:
df.to_csv('../Data/data_clean.csv', index=False)